# Identify All API Calls


In [1]:
import pandas as pd
import json
import os
from collections import defaultdict
import re
import random

In [2]:
"""
there are 3 API patters
http://localhost:3000/api/v1/master
http://latest-main.determined.ai:8080/api/v1/master
http://latest-ee-gke-main.determined.ai/api/v1/master
"""
API_PATTERN = r"^http://(latest-(main|ee-gke-main)\.determined\.ai|localhost)(:8080|:3000)?/api"

def get_file(trace_list) -> list[str]:
    api_set = set()
    for obj in trace_list:
        try:
            data_url = str(obj["args"]["url"])
            if re.match(API_PATTERN, data_url, re.IGNORECASE):
                api_set.add(re.sub(API_PATTERN, "", data_url, re.IGNORECASE))
                continue
        except:
            pass
        try:
            data_url = str(obj["args"]["data"]["url"])
            if re.match(API_PATTERN, data_url, re.IGNORECASE):
                api_set.add(re.sub(API_PATTERN, "", data_url, re.IGNORECASE))
        except:
            pass
    return sorted(list(api_set))

In [3]:
files = sorted(os.listdir("data"))
api_dict: dict[str, list[str]] = defaultdict(list)
for file in files:
    if not file.lower().endswith("json"):
        continue
    try:
        json_open = open(f"./data/{file}", "r")
        json_load = json.load(json_open)
        trace_list = json_load["traceEvents"]
        api_dict["url"] += [file.split(".")[0].replace(":", "/")]
        api_dict["api"] += [get_file(trace_list)]
        api_dict["root_url"] += [file.split(":")[0]]
    except:
        print(file)
        break

In [4]:
def generate_colors() -> dict[str, str]:
    color_dict = {}
    for i, file in enumerate(files):
        if not file.lower().endswith("json"):
            continue
        random.seed(i * 10)
        color = "%06x" % random.randint(0, 0xFFFFFF)
        color_dict[file.split(".")[0].replace(":", "/")] = f"#{color}97"
    return color_dict


generate_colors()

{'admin/group-management': '#10aefd97',
 'admin/user-management': '#4d6ac797',
 'clusters': '#94118e97',
 'clusters/historical-usage': '#eace7f97',
 'clusters/logs': '#febd1c97',
 'dashboard': '#9d98d897',
 'experiments/{experiment_id}/checkpoints': '#3ca35297',
 'experiments/{experiment_id}/code': '#8b00e297',
 'experiments/{experiment_id}/hyperparameters': '#68477e97',
 'experiments/{experiment_id}/logs': '#4a952997',
 'experiments/{experiment_id}/metrics': '#c6ac7c97',
 'experiments/{experiment_id}/notes': '#7eeb4097',
 'experiments/{experiment_id}/overview': '#d9d67597',
 'experiments/{experiment_id}/profiler': '#30a57897',
 'experiments/{experiment_id}/trials': '#a7197797',
 'experiments/{experiment_id}/trials/{trial_id}/hyperparameters': '#3ebd2b97',
 'experiments/{experiment_id}/trials/{trial_id}/metrics': '#b0a24697',
 'experiments/{experiment_id}/trials/{trial_id}/profiler': '#49d9cc97',
 'experiments/{experiment_id}/trials/{trials_id}/overview': '#26304d97',
 'experiments/{ex

In [5]:
pd.set_option("display.max_colwidth", 5000)
df = pd.DataFrame.from_dict(api_dict)
df_collapse = df.explode("api")

In [6]:
df

,url,api,root_url
0,admin/group-management,"[/v1/agents, /v1/auth/user, /v1/groups/178, /v1/groups/180, /v1/groups/183, /v1/groups/185, /v1/groups/191, /v1/groups/206, /v1/groups/207, /v1/groups/209, /v1/groups/220, /v1/groups/3, /v1/groups/4, /v1/groups/5, /v1/groups/71, /v1/groups/77, /v1/groups/80, /v1/groups/86, /v1/groups/92, /v1/groups/search, /v1/master, /v1/permissions/summary, /v1/resource-pools, /v1/roles/search, /v1/roles/search/by-group/178, /v1/roles/search/by-group/180, /v1/roles/search/by-group/183, /v1/roles/search/by-group/185, /v1/roles/search/by-group/191, /v1/roles/search/by-group/206, /v1/roles/search/by-group/207, /v1/roles/search/by-group/209, /v1/roles/search/by-group/220, /v1/roles/search/by-group/3, /v1/roles/search/by-group/4, /v1/roles/search/by-group/5, /v1/roles/search/by-group/71, /v1/roles/search/by-group/77, /v1/roles/search/by-group/80, /v1/roles/search/by-group/86, /v1/roles/search/by-group/92, /v1/users, /v1/users/setting, /v1/workspaces]",admin
1,admin/user-management,"[/v1/agents, /v1/auth/user, /v1/groups/search, /v1/master, /v1/permissions/summary, /v1/resource-pools, /v1/roles/search, /v1/roles/search/by-user/1, /v1/roles/search/by-user/10, /v1/roles/search/by-user/11, /v1/roles/search/by-user/3, /v1/roles/search/by-user/4, /v1/roles/search/by-user/5, /v1/roles/search/by-user/6, /v1/roles/search/by-user/7, /v1/roles/search/by-user/8, /v1/roles/search/by-user/9, /v1/users, /v1/users/setting, /v1/workspaces]",admin
2,clusters,"[/v1/agents, /v1/auth/user, /v1/experiments?limit=-2&states=STATE_ACTIVE&states=STATE_STOPPING_CANCELED&states=STATE_STOPPING_COMPLETED&states=STATE_STOPPING_ERROR&projectId=0&showTrialData=true, /v1/master, /v1/master/telemetry, /v1/resource-pools, /v1/resource-pools/aux-pool/workspace-bindings, /v1/resource-pools/compute-pool/workspace-bindings, /v1/tasks/count, /v1/users, /v1/users/setting, /v1/workspaces]",clusters.json
3,clusters/historical-usage,"[/v1/agents, /v1/auth/user, /v1/master, /v1/master/telemetry, /v1/resource-pools, /v1/resources/allocation/aggregated?startDate=2023-08-03&endDate=2023-08-22&period=RESOURCE_ALLOCATION_AGGREGATION_PERIOD_DAILY, /v1/resources/allocation/aggregated?startDate=2023-08-08&endDate=2023-08-22&period=RESOURCE_ALLOCATION_AGGREGATION_PERIOD_DAILY, /v1/users, /v1/users/setting, /v1/workspaces]",clusters
4,clusters/logs,"[/v1/agents, /v1/auth/user, /v1/master, /v1/master/logs?offset=-1&limit=0&follow=true, /v1/master/logs?offset=-100&limit=100&follow=false, /v1/master/telemetry, /v1/resource-pools, /v1/users, /v1/users/setting, /v1/workspaces]",clusters
5,dashboard,"[/v1/agents, /v1/auth/user, /v1/commands?sortBy=SORT_BY_START_TIME&orderBy=ORDER_BY_DESC&limit=25&userIds=1, /v1/experiments?sortBy=SORT_BY_START_TIME&orderBy=ORDER_BY_DESC&limit=25&userIds=1&projectId=0&showTrialData=true, /v1/master, /v1/master/telemetry, /v1/notebooks?sortBy=SORT_BY_START_TIME&orderBy=ORDER_BY_DESC&limit=25&userIds=1, /v1/resource-pools, /v1/shells?sortBy=SORT_BY_START_TIME&orderBy=ORDER_BY_DESC&limit=25&userIds=1, /v1/tensorboards?sortBy=SORT_BY_START_TIME&orderBy=ORDER_BY_DESC&limit=25&userIds=1, /v1/user/projects/activity?limit=5, /v1/users, /v1/users/setting, /v1/workspaces]",dashboard.json
6,experiments/{experiment_id}/checkpoints,"[/v1/agents, /v1/auth/user, /v1/experiments/3829, /v1/experiments/3829/checkpoints?sortBy=SORT_BY_UUID&orderBy=ORDER_BY_DESC&offset=0&limit=10, /v1/experiments/3829/trials?limit=2, /v1/master, /v1/master/telemetry, /v1/resource-pools, /v1/trials/34667, /v1/users, /v1/users/setting, /v1/workspaces]",experiments
7,experiments/{experiment_id}/code,"[/v1/agents, /v1/auth/user, /v1/experiments/3829, /v1/experiments/3829/file, /v1/experiments/3829/file_tree, /v1/experiments/3829/trials?limit=2, /v1/master, /v1/master/telemetry, /v1/resource-pools, /v1/trials/34667, /v1/users, /v1/users/setting, /v1/workspaces]",experiments
8,experiments/{experiment_id}/hyperparameters,"[/v1/agents, /v1/auth/user, /v1/experiments/3829, /v1

In [7]:
df_collapse = df_collapse.reset_index(drop=True)
df_collapse = df_collapse.drop(columns=['root_url'])

In [8]:
df_collapse

,url,api
0,admin/group-management,/v1/agents
1,admin/group-management,/v1/auth/user
2,admin/group-management,/v1/groups/178
3,admin/group-management,/v1/groups/180
4,admin/group-management,/v1/groups/183
...,...,...
558,workspaces/{workspace_id}/tasks,/v1/shells?limit=1000&workspaceId=2
559,workspaces/{workspace_id}/tasks,/v1/tensorboards?limit=1000&workspaceId=2
560,workspaces/{workspace_id}/tasks,/v1/users
561,workspaces/{workspace_id}/tasks,/v1/users/setting


Output HTML with some styling

In [9]:
HTML_OUTPUT = "./stats.html"
color_dict = generate_colors()

with open(HTML_OUTPUT, mode="w") as f:
    f.write(
        df_collapse.style.applymap(
            lambda x: f'background-color: {color_dict.get(x, "")}', subset=["url"]
        )
        .set_table_styles(
            [
                {
                    "selector": "tbody tr:nth-child(even)",
                    "props": [("background-color", "#ededed")],
                },
                {
                    "selector": "tr > th, tr > td",
                    "props": [("border", "1px solid gray"), ("padding", "4px 8px")],
                },
                {
                    "selector": "",
                    "props": [("border-collapse", "collapse")],
                }
            ]
        )
        .to_html(columns=["url", "api"], justify="left")
    )